In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.common.exceptions import TimeoutException
import pandas as pd
import numpy as np
import datetime
import time
from itertools import zip_longest
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe 
import seaborn as sns
import googlemaps
import requests
import re
import os
from geopy.geocoders import Nominatim
from PIL import Image
import imagehash


In [17]:
navegador = webdriver.Chrome()  ## iniciando o webdriver do navegador

todos_os_dados = []  ## criando lista vazia para armazenar os dados de diferentes pa´ginas

pagina = 1  ## contador de páginas sendo atribuído para 1

def baixar_imagem(url, pasta_destino="imagens"):
    os.makedirs(pasta_destino, exist_ok=True)
    nome_arquivo = url.split("/")[-1]
    caminho = os.path.join(pasta_destino, nome_arquivo)

    resposta = requests.get(url)
    with open(caminho, "wb") as f:
        f.write(resposta.content)

    return caminho

while True:
    #utilizei a URL com filtra de imóveis para venda.
    url = f"https://www.olx.com.br/imoveis/venda/estado-rj/serra-angra-dos-reis-e-regiao/petropolis?sf=1&o={pagina}"  ## esse valor entre páginas serve para identificar em qual página o WebDriver está lendo e por ser um contador, a cada loop uma nova página é lida
    print(f"\nAcessando página {pagina}: {url}")
    navegador.get(url)
    
    #Verifica se ainda encontra anuncios na página (Estava parando de encontrar na página 101, porém continuando a pesquisa indefinidamente.)
    try:
        navegador.find_element(By.CSS_SELECTOR, "span.typo-title-large")
        print("Nenhum anúncio encontrado nesta página. Encerrando.")
        break
    except:
        # não encontrou a mensagem, continua a raspagem
        pass

    try:
        WebDriverWait(navegador, 12).until(ec.presence_of_all_elements_located((
        By.CSS_SELECTOR, 'h3.typo-body-large.olx-adcard__price.font-semibold'
        )))  ## Adicionando comando de espera para garantir que todos os elementos da página foram devidamente carregados

        elementos_valores = navegador.find_elements(By.CSS_SELECTOR, 'h3.typo-body-large.olx-adcard__price.font-semibold')   ## localizando os elementos de preco
        elementos_localizacoes = navegador.find_elements(By.CSS_SELECTOR, 'p.olx-adcard__location')   ## localizando os elementos de localização
        elementos_datas = navegador.find_elements(By.CSS_SELECTOR, 'p.typo-caption.olx-adcard__date')   ## localizando os elementos de data
        anuncios = navegador.find_elements(By.CSS_SELECTOR, 'section.olx-adcard.olx-adcard__horizontal.undefined')
                 
        lista_valores = [valor.text.strip() for valor in elementos_valores]  ## gerando a lista com os valores dos imóveis de uma página
        lista_localizacoes = [loc.text.strip() for loc in elementos_localizacoes] ## gerando a lista com as localizações dos imóveis de uma página
        lista_data_anuncio = [data.text.strip() for data in elementos_datas]


        lista_quartos = [] ## gerando a lista com qtd de qwuartos dos imóveis de uma página
        lista_areas = [] ## gerando lista com areas
        lista_vagas = []
        lista_banheiros = []
        lista_iptu = []
        lista_condominio = []
        lista_links = []
        lista_imagens = []

        for anuncio in anuncios:
            detalhes = anuncio.find_elements(By.CSS_SELECTOR, "div.olx-adcard__detail")
            precos = anuncio.find_elements(By.CSS_SELECTOR, "div.olx-adcard__price-info")
            links = anuncio.find_elements(By.CSS_SELECTOR, "a.olx-adcard__link")
            elementos_imagens = anuncio.find_elements(By.CSS_SELECTOR, "picture img")
            area = ""
            quartos = ""
            vagas = ""
            banheiros = ""
            iptu = ""
            condominio = ""
            link = ""
            imagem = ""
            
            for url in elementos_imagens:
                url_imagem = url.get_attribute("src")
                imagem = baixar_imagem(url_imagem)

            for l in links: #extraindo links dos anuncios
                link = l.get_attribute("href")

            for preco in precos:
                if "IPTU" in preco.text.strip():
                    match = re.search(r"[\d\.,]+", preco.text) ## extrai os números com pontos
                    if match:
                        iptu = match.group()
                    
                elif "Condomínio" in preco.text.strip():
                    match = re.search(r"[\d\.,]+", preco.text) ## extrai os números com pontos
                    if match:
                        condominio = match.group()

            for d in detalhes:
                info = d.get_attribute("aria-label")  # extrai direto o atributo
                if info:
                    info_lower = info.lower()
                    if "metro" in info_lower or "m²" in info_lower:
                        match = re.search(r"\d+", info) ## extrai apenas os números
                        if match:
                            area = int(match.group())                        
                    elif "quarto" in info_lower:
                        match = re.search(r"\d+", info) ## Extrai apenas os números
                        if match:
                            quartos = int(match.group())
                    elif "vaga" in info_lower:
                        match = re.search(r"\d+", info) ## Extrai apenas os números
                        if match:
                            vagas = int(match.group())
                    elif "banheiro" in info_lower:
                        match = re.search(r"\d+", info) ## Extrai apenas os números
                        if match:
                            banheiros = int(match.group())

            lista_areas.append(area)
            lista_quartos.append(quartos)
            lista_vagas.append(vagas)
            lista_banheiros.append(banheiros)
            lista_iptu.append(iptu)
            lista_condominio.append(condominio)
            lista_links.append(link)
            lista_imagens.append(imagem)

        if not lista_valores:  ## se não há valores na página, não ocorre raspagem
            print("Sem valores nesta página. Fim da raspagem.")
            break

        df_valores = pd.DataFrame(
            list(zip_longest(lista_valores, lista_localizacoes, lista_areas, lista_quartos, lista_vagas, lista_banheiros, lista_iptu, lista_condominio, lista_data_anuncio, lista_links, lista_imagens, fillvalue="")), ## garantindo que os espaços vazios nas lista sejam preenchidos com ""
        columns=["valor", "localizacao", "m2", "quartos", "vagas", "banheiros", "iptu", "condominio", "data_anuncio", "link", "imagem"])
        df_valores["data"] = datetime.date.today()  ## gerando coluna com a data de extração dos registros

        todos_os_dados.append(df_valores)  ## juntando todos os dados extraídos
        print(f"Total extraído nesta página: {len(df_valores)}")

        pagina += 1  ## a cada loop, o contador de página assume +1 e a url é alterada, fazendo com que uma nova página seja lida
        time.sleep(2) ## aguarda 2 segundos para começar o novo loop

    except TimeoutException:
        print("Nenhum anúncio encontrado ou página demorou a carregar. Encerrando.")
        break   ## se ocorrer o erro Timeouexcpetion, o loop é encerrado

df_final = pd.concat(todos_os_dados, ignore_index=True)  ## juntando todos os dados obtidos de todas as páginas lidas
print(f"\nTotal geral de valores extraídos: {len(df_final)}")
print(df_final)

navegador.quit()



Acessando página 1: https://www.olx.com.br/imoveis/venda/estado-rj/serra-angra-dos-reis-e-regiao/petropolis?sf=1&o=1
Total extraído nesta página: 53

Acessando página 2: https://www.olx.com.br/imoveis/venda/estado-rj/serra-angra-dos-reis-e-regiao/petropolis?sf=1&o=2
Total extraído nesta página: 50

Acessando página 3: https://www.olx.com.br/imoveis/venda/estado-rj/serra-angra-dos-reis-e-regiao/petropolis?sf=1&o=3
Total extraído nesta página: 50

Acessando página 4: https://www.olx.com.br/imoveis/venda/estado-rj/serra-angra-dos-reis-e-regiao/petropolis?sf=1&o=4
Total extraído nesta página: 50

Acessando página 5: https://www.olx.com.br/imoveis/venda/estado-rj/serra-angra-dos-reis-e-regiao/petropolis?sf=1&o=5
Total extraído nesta página: 50

Acessando página 6: https://www.olx.com.br/imoveis/venda/estado-rj/serra-angra-dos-reis-e-regiao/petropolis?sf=1&o=6
Total extraído nesta página: 50

Acessando página 7: https://www.olx.com.br/imoveis/venda/estado-rj/serra-angra-dos-reis-e-regiao/pe

In [18]:
lista_areas

[483,
 441,
 180,
 1060,
 57,
 149,
 200,
 600,
 220,
 695,
 44,
 200,
 367,
 300,
 126,
 500,
 700,
 234,
 106,
 100,
 70,
 44,
 100,
 2395,
 235,
 300,
 5000,
 38,
 31,
 77,
 207,
 72,
 92,
 350,
 53,
 140,
 250,
 1800,
 170,
 93,
 400,
 150,
 120,
 10,
 56,
 70,
 82,
 82,
 130,
 90]

In [19]:
df_final[['cidade','bairro']] = df_final['localizacao'].str.split(", ", n=1, expand=True)
df_final = df_final.drop(columns='localizacao')

In [20]:
df_final.isnull().sum()

valor           0
m2              0
quartos         0
vagas           0
banheiros       0
iptu            0
condominio      0
data_anuncio    0
link            0
imagem          0
data            0
cidade          0
bairro          2
dtype: int64

In [21]:
df_final.dropna(subset='bairro', inplace=True)

In [22]:
index_drop = df_final.loc[df_final['m2'] == ""].index
df_final = df_final.drop(index_drop)

In [23]:
#Converte data_anuncio para formato datetime

def converter_data(texto):
    agora = datetime.datetime.now()
    if 'Hoje' in texto:
        hora = texto.split(',')[1].strip()
        return datetime.datetime.strptime(f"{agora.date()} {hora}", "%Y-%m-%d %H:%M")
    elif 'Ontem' in texto:
        hora = texto.split(',')[1].strip()
        ontem = agora.date() - datetime.timedelta(days=1)
        return datetime.datetime.strptime(f"{ontem} {hora}", "%Y-%m-%d %H:%M")
    else:
        dia, hora = texto.split(', ')
        dia_num, mes_abrev = dia.split(' de ')
        mes_num = {
            'jan': 1, 'fev': 2, 'mar': 3, 'abr': 4, 'mai': 5, 'jun': 6,
            'jul': 7, 'ago': 8, 'set': 9, 'out': 10, 'nov': 11, 'dez': 12
        }[mes_abrev]
        ano = agora.year
        return datetime.datetime.strptime(f"{dia_num}/{mes_num}/{ano} {hora}", "%d/%m/%Y %H:%M")

df_final["data_anuncio"] = df_final["data_anuncio"].apply(converter_data)

In [24]:
#Transforma valor em Número (intt)
# limpeza inicial
df_final["valor"] = (
    df_final["valor"]
    .str.replace("R$", "", regex=False)   # remove R$
    .str.replace(".", "", regex=False)    # remove separador de milhar
    .str.replace(",", ".", regex=False)   # troca vírgula por ponto
    .str.strip()
)

# converte para número (mantém NaN nos inválidos)
df_final["valor"] = pd.to_numeric(df_final["valor"], errors="coerce")

# converte para inteiro, removendo os NaN antes
df_final["valor"] = df_final["valor"].dropna().astype(int)

In [25]:
# inicializa o geocodificador
geolocator = Nominatim(user_agent="meu_app")

# cache (dicionário)
geocode_cache = {}

def geocode_location_osm(bairro, cidade):
    endereco = f"{bairro}, {cidade}, RJ, Brasil"
    
    # se já foi consultado antes, retorna do cache
    if endereco in geocode_cache:
        return geocode_cache[endereco]
    
    # senão, consulta no Nominatim
    try:
        location = geolocator.geocode(endereco, timeout=10)
        time.sleep(1)  # respeita o limite de 1 req/segundo
        if location:
            resultado = (location.latitude, location.longitude)
        else:
            resultado = (None, None)
    except Exception as e:
        print(f"Erro ao geocodificar {endereco}: {e}")
        resultado = (None, None)
    
    # guarda no cache
    geocode_cache[endereco] = resultado
    return resultado

# aplicar no DataFrame
df_final[['latitude', 'longitude']] = df_final.apply(
    lambda row: pd.Series(geocode_location_osm(row['bairro'], row['cidade'])),
    axis=1
)


In [26]:
#Excluir anuncios duplicadas (Comparando as imagens dos anuncios)

# Função para gerar hash visual
def gerar_hash_imagem(caminho_imagem):
    try:
        imagem = Image.open(caminho_imagem)
        return imagehash.average_hash(imagem)
    except Exception as e:
        print(f"Erro ao processar {caminho_imagem}: {e}")
        return None

# Gera os hashes
df_final['hash_imagem'] = df_final['imagem'].apply(gerar_hash_imagem)
df_final['hash_str'] = df_final['hash_imagem'].astype(str)

# Cria cópia antes de filtrar
df_completo = df_final.copy()

# Identifica grupos com hash duplicado
grupo_duplicados = df_completo[df_completo.duplicated(subset='hash_str', keep=False)]

# Filtra os que têm imagem igual E atributos iguais
duplicados_confirmados = grupo_duplicados[
    grupo_duplicados.duplicated(subset=['hash_str', 'bairro', 'quartos', 'm2'], keep=False)
]

# Remove os duplicados confirmados, mantendo o primeiro
df_unicos = df_completo.drop(duplicados_confirmados.index[1:])  # mantém o primeiro de cada grupo

# Exibe os grupos considerados duplicados
grupos = duplicados_confirmados.sort_values('hash_str').groupby('hash_str')
for hash_valor, grupo in grupos:
    print(f"\n🔁 Hash duplicado + atributos iguais: {hash_valor}")
    print(grupo[['valor', 'm2', 'quartos', 'bairro', 'imagem']])

Erro ao processar : 'str' object has no attribute 'read'
Erro ao processar : 'str' object has no attribute 'read'
Erro ao processar : 'str' object has no attribute 'read'
Erro ao processar : 'str' object has no attribute 'read'
Erro ao processar : 'str' object has no attribute 'read'
Erro ao processar : 'str' object has no attribute 'read'
Erro ao processar : 'str' object has no attribute 'read'

🔁 Hash duplicado + atributos iguais: 00008080187cfeff
         valor   m2 quartos    bairro                        imagem
2565  12000000  868       5  Itaipava  imagens\490548568921845.webp
1570  12000000  868       5  Itaipava  imagens\711544322965621.webp

🔁 Hash duplicado + atributos iguais: 0000ffffe7f73c35
        valor   m2 quartos  bairro                        imagem
21    2100000  344       4  Centro  imagens\583573803342050.webp
1706  2100000  344       4  Centro  imagens\582599315746384.webp
2228  2100000  344       4  Centro  imagens\856580687682981.webp

🔁 Hash duplicado + atribut

In [27]:
df_final

,valor,m2,quartos,vagas,banheiros,iptu,condominio,data_anuncio,link,imagem,data,cidade,bairro,latitude,longitude,hash_imagem,hash_str
0,455000,70,2,1,2,332,751,2025-09-26 21:26:00,https://rj.olx.com.br/serra-angra-dos-reis-e-r...,imagens\809548138667735.webp,2025-09-27,Petrópolis,Itaipava,-22.412700,-43.142100,ffde00000040df7f,ffde00000040df7f
1,430000,56,2,1,1,110,210,2025-09-26 11:12:00,https://rj.olx.com.br/serra-angra-dos-reis-e-r...,imagens\119505194936684.webp,2025-09-27,Petrópolis,Corrêas,-22.442565,-43.139706,ffff7f5330202010,ffff7f5330202010
3,150000,62,1,,1,,,2025-09-25 14:37:00,https://rj.olx.com.br/serra-angra-dos-reis-e-r...,imagens\686564086666094.webp,2025-09-27,Petrópolis,Quissama,-22.492869,-43.159080,800002dee0e0fcfc,800002dee0e0fcfc
4,1950000,157,4,3,1,389,1.010,2025-09-27 13:53:00,https://rj.olx.com.br/serra-angra-dos-reis-e-r...,imagens\617521442792009.webp,2025-09-27,Petrópolis,Araras,-22.425156,-43.221496,ddd0ac0337721f86,ddd0ac0337721f86
5,4200000,4600,4,2,3,300,1.560,2025-09-27 13:53:00,https://rj.olx.com.br/serra-angra-dos-reis-e-r...,imagens\621572563578972.webp,2025-09-27,Petrópolis,Araras,-22.425156,-43.221496,9093110000674fff,9093110000674fff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4998,520000,70,2,1,1,197,577,2025-01-14 05:47:00,https://rj.olx.com.br/serra-angra-dos-reis-e-r...,imagens\729574445095117.webp,2025-09-27,Petrópolis,Itaipava,-22.412700,-43.142100,3f3f3f1f1f0f0e0e,3f3f3f1f1f0f0e0e
4999,586000,82,3,2,2,,,2025-01-14 04:02:00,https://rj.olx.com.br/serra-angra-dos-reis-e-r...,imagens\604503449593246.webp,2025-09-27,Petrópolis,Corrêas,-22.442565,-43.139706,f0fdfcf0f6a0801f,f0fdfcf0f6a0801f
5000,589000,82,3,2,2,,,2025-01-14 03:56:00,https://rj.olx.com.br/serra-angra-dos-reis-e-r...,imagens\515575680344275.webp,2025-09-27,Petrópolis,Corrêas,-22.442565,-43.139706,fe0000fc21fff386,fe0000fc21fff386
5001,720000,130,3,2,3,,,2025-01-14 03:56:00,https://rj.olx.com.br/serra-angra-dos-reis-e-r...,imagens\396565442889799.webp,2025-09-27,Petrópolis,Mosela,-22.493190,-43.202826,fefc40001c1d9fff,fefc40001c1d9fff


In [28]:
# Salvar em .csv

# gera a data de hoje no formato YYYY-MM-DD
data_hoje = datetime.date.today().strftime("%Y-%m-%d")

# monta o nome do arquivo
nome_arquivo = f"imoveis_petropolis_{data_hoje}.csv"

df_final["valor"] = df_final["valor"].fillna(0).astype(int)

# salva o DataFrame
df_final.to_csv(nome_arquivo, index=False, sep=";", encoding="utf-8-sig")

print(f"Arquivo salvo: {nome_arquivo}")
print(os.getcwd())  # mostra o diretório onde o arquivo foi salvo

Arquivo salvo: imoveis_petropolis_2025-09-27.csv
e:\Ciencia de dados\PAE\analise_imobiliaria_petropolitana
